In [0]:
server_name = "jdbc:sqlserver://suparna07.database.windows.net"
database_name = "AdventureWorks"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "Sample07"
username = ""
password = ""

try:
    DF_SQLDB_Read = (spark.read \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .option("url" , url) \
    .option("dbtable" , table_name) \
    .option("user" , username) \
    .option("password" , password) \
    .load())
    display(DF_SQLDB_Read)
except ValueError as e:
    print("Cannot connect with the DB")

Username,Identifier,First name,Last name,id
johnson81,4081,Craig,Johnson,1
jenkins46,9346,Mary,Jenkins,2
smith79,5079,Jamie,Smith,3
johnson81,4081,Craig,Johnson,0
johnson81,4081,Craig,Johnson,1
jenkins46,9346,Mary,Jenkins,2
smith79,5079,Jamie,Smith,3


In [0]:
from pyspark.sql.functions import monotonically_increasing_id
# print(DF_SQLDB_Read.count())
DF_SQLDB_Read_With_ID = DF_SQLDB_Read.withColumn("id" , monotonically_increasing_id().cast("string"))
display(DF_SQLDB_Read_With_ID)

Username,Identifier,First name,Last name,id
johnson81,4081,Craig,Johnson,0
jenkins46,9346,Mary,Jenkins,1
smith79,5079,Jamie,Smith,2
johnson81,4081,Craig,Johnson,3
johnson81,4081,Craig,Johnson,4
jenkins46,9346,Mary,Jenkins,5
smith79,5079,Jamie,Smith,6


In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number


DF_AzureSQLDB_Upsert = spark.createDataFrame(DF_SQLDB_Read_With_ID.tail(4))
DF_AzureSQLDB_Upsert = DF_AzureSQLDB_Upsert.withColumn("counter", monotonically_increasing_id())
w = Window.orderBy("counter")
# Use row number with the window specification
DF_AzureSQLDB_Upsert = DF_AzureSQLDB_Upsert.withColumn("id", row_number().over(w).cast("string"))
DF_AzureSQLDB_Upsert = DF_AzureSQLDB_Upsert.drop('counter')
display(DF_AzureSQLDB_Upsert)

Username,Identifier,First name,Last name,id
johnson81,4081,Craig,Johnson,1
johnson81,4081,Craig,Johnson,2
jenkins46,9346,Mary,Jenkins,3
smith79,5079,Jamie,Smith,4


In [0]:
from pyspark.sql.functions import col

DF_SQLDB_Read_With_ID_OLD = DF_SQLDB_Read_With_ID.alias("target") \
    .join(DF_AzureSQLDB_Upsert.alias("source"), \
        col("source.id") == col("target.id") ,
        how='leftanti')
    
DF_Final = DF_SQLDB_Read_With_ID_OLD.union(DF_AzureSQLDB_Upsert).sort('id')
    
display(DF_Final)

Username,Identifier,First name,Last name,id
johnson81,4081,Craig,Johnson,0
johnson81,4081,Craig,Johnson,1
johnson81,4081,Craig,Johnson,2
jenkins46,9346,Mary,Jenkins,3
smith79,5079,Jamie,Smith,4
jenkins46,9346,Mary,Jenkins,5
smith79,5079,Jamie,Smith,6


In [0]:
%sh
if ! [[ "16.04 18.04 20.04 22.04" == *"$(lsb_release -rs)"* ]];
then
    echo "Ubuntu $(lsb_release -rs) is not currently supported.";
    exit;
fi

curl https://packages.microsoft.com/keys/microsoft.asc | sudo tee /etc/apt/trusted.gpg.d/microsoft.asc

curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list | sudo tee /etc/apt/sources.list.d/mssql-release.list

sudo apt-get update
sudo ACCEPT_EULA=Y apt-get install -y msodbcsql17
# optional: for bcp and sqlcmd
sudo ACCEPT_EULA=Y apt-get install -y mssql-tools
echo 'export PATH="$PATH:/opt/mssql-tools/bin"' >> ~/.bashrc
source ~/.bashrc
# optional: for unixODBC development headers
sudo apt-get install -y unixodbc-dev

In [0]:
import pyodbc

server = 'suparna07.database.windows.net,1433'
database = database_name
username = username
password = password

conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'

conn =pyodbc.connect(conn_str)
cursor = conn.cursor()


drop_query = "Drop table dbo.Sample07"
cursor.execute(drop_query)
conn.commit()

In [0]:
host = "suparna07.database.windows.net"
port = "1433"
database = "AdventureWorks"

numberCores = spark.sparkContext.defaultParallelism


try:
 DF_Final.write \
   .mode("default") \
   .format("sqlserver") \
   .option("host" , host) \
   .option("port" , port) \
   .option("databasename" , database) \
   .option("dbtable" , table_name) \
   .option("user",username) \
   .option("password",password) \
   .option("numPartitions", numberCores) \
   .option("tableLock", "true") \
   .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver" ) \
   .save()
except ValueError as error :
    print("Connector write failed", error)